In [29]:
import helper
import importlib
importlib.reload(helper)
import pandas as pd 
import numpy as np
from sklearn.pipeline import make_pipeline

In [30]:
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.model_selection import train_test_split

In [31]:
#import breast dataset
X = pd.read_csv("./datasets/cleaned/breast_X.csv")
X = X.drop("Unnamed: 0", axis=1)
y = pd.read_csv("./datasets/cleaned/breast_y.csv")
y = y.drop("Unnamed: 0", axis=1)

X.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [32]:
y.head()

,diagnosis
0,1
1,1
2,1
3,1
4,1


In [33]:
features_types_df = pd.read_csv("datasets/cleaned/datatypes/breast.csv")
features_types_df.T

,0,1
0,radius_mean,0
1,texture_mean,0
2,perimeter_mean,0
3,area_mean,0
4,smoothness_mean,0
5,compactness_mean,0
6,concavity_mean,0
7,concave points_mean,0
8,symmetry_mean,0
9,fractal_dimension_mean,0


In [34]:
feature_inidices = list(map(int, list(features_types_df)))

In [35]:
features_names = list(features_types_df.T[0])

In [36]:
features_types = list(map(int, list(features_types_df.T[1])))

In [37]:
preprocess = helper.select_preprocessing_for_many_feat(feature_inidices, features_types, features_names)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1337, shuffle=True)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1337)

adult_ebm = make_pipeline(
    preprocess,
    ExplainableBoostingClassifier()
)

adult_ebm.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(), []),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  ['radius_mean',
                                                   'texture_mean',
                                                   'perimeter_mean',
                                                   'area_mean',
                                                   'smoothness_mean',
                                                   'compactness_mean',
                                                   'concavity_mean',
                                                   'concave points_mean',
                                                   'symmetry_mean',
                                                   'fractal_dimension

In [39]:
adult_ebm.score(X_val, y_val)

0.9418604651162791

In [40]:
y_preds = adult_ebm.predict(X_val)

In [41]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_val, y_preds)
metrics.auc(fpr, tpr)

0.9321428571428572